# Tarefa 03

- Leia os enunciados com atenção
- Saiba que pode haver mais de uma resposta correta
- Insira novas células de código sempre que achar necessário
- Em caso de dúvidas, procure os Tutores
- Divirta-se :)

In [4]:
import pandas as pd
import requests

In [10]:
url = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl' #guardando erl dentro de variavel

In [26]:
r = requests.get(url) #consultando status http

In [32]:
requests.get(url).json()["items"] #consultando o status http, pegando o json e selecionando a lista items

[{'id_pleito': 30070,
  'tipo_interessado': 'Município',
  'interessado': 'Maçambará',
  'cod_ibge': 4311718,
  'uf': 'RS',
  'num_pvl': 'PVL02.002523/2018-16',
  'status': 'Deferido (PVL-IF)',
  'num_processo': None,
  'data_protocolo': None,
  'tipo_operacao': 'Operação contratual interna',
  'finalidade': 'Aquisição de máquinas, equipamentos e veículos',
  'tipo_credor': 'Instituição Financeira Nacional',
  'credor': 'Banco do Estado do Rio Grande do Sul',
  'moeda': 'Real',
  'valor': 2000000,
  'pvl_assoc_divida': 1,
  'pvl_contradado_credor': 1,
  'data_status': '19/10/2018'},
 {'id_pleito': 29616,
  'tipo_interessado': 'Município',
  'interessado': 'Maripá de Minas',
  'cod_ibge': 3140209,
  'uf': 'MG',
  'num_pvl': 'PVL02.002140/2018-30',
  'status': 'Deferido (PVL-IF)',
  'num_processo': None,
  'data_protocolo': None,
  'tipo_operacao': 'Operação contratual interna',
  'finalidade': 'Aquisição de máquinas, equipamentos e veículos',
  'tipo_credor': 'Instituição Financeira Nac

####  1) Lendo de APIs
Vimos em aula como carregar dados públicos do governo através de um API (*Application Programming Interface*). No exemplo de aula, baixamos os dados de pedidos de verificação de limites (PVL) realizados por estados, e selecionamos apenas aqueles referentes ao estado de São Paulo.

1. Repita os mesmos passos feitos em aula, mas selecione os PVLs realizados por municípios no estado do Rio de Janeiro.
2. Quais são os três *status* das solicitações mais frequentes na base? Quais são suas frequências?
3. Construa uma nova variável que contenha o ano do **status**. Observe que ```data_status``` vem como tipo *object* no **DataFrame**. Dica: você pode usar o método ```.str``` para transformar o tipo da variável em string, em seguida um método como [**slice()**](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.slice.html) ou [**split()**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.split.html).
4. Indique a frequência de cada ano do campo construído no item (3).

In [236]:
# 1) Seu código aqui
df = pd.DataFrame(requests.get('https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl').json()["items"])
df1 = df[df['uf'] == 'RJ'].reset_index()
df1.dtypes
df1[df1['tipo_interessado'] == 'Município']['num_pvl'].unique()

array(['PVL02.009440/2019-21', 'PVL02.001389/2018-28',
       'PVL02.002429/2017-78', 'PVL02.002481/2019-96', None,
       'PVL02.000145/2016-66', 'PVL02.003090/2019-99',
       'PVL02.005421/2019-25', 'PVL02.009521/2019-21',
       '00000.000000/2032-87', 'PVL02.000644/2024-63',
       'PVL02.002586/2023-21', 'PVL02.005942/2022-88',
       'PVL02.000206/2023-14', 'PVL02.006783/2022-39',
       'PVL02.000025/2023-98', 'PVL02.000708/2019-69',
       'PVL02.003091/2019-33', 'PVL02.000583/2019-77'], dtype=object)

In [238]:
# 2) Seu código aqui
status = list(df1.status.value_counts().head(3).index)
valores = list(df1.status.value_counts().head(3))

status_tuples = list(zip(status, valores))
status_tuples

[('Deferido', 29),
 ('Encaminhado à PGFN com manifestação técnica favorável', 13),
 ('Arquivado', 10)]

In [265]:
# 3) Seu código aqui
df1['data_status'] = df1['data_status'].astype(str)
df1['ano_status'] = df1['data_status'].str.split('/').str[2]
df1[['ano_status','data_status']]

,ano_status,data_status
0,2020,09/03/2020
1,2018,12/07/2018
2,2019,07/01/2019
3,2019,31/07/2019
4,2013,26/12/2013
...,...,...
65,2014,11/06/2014
66,2011,04/07/2011
67,2019,23/04/2019
68,2007,26/06/2007


In [267]:
#4

df1.ano_status.value_counts()

ano_status
2008    10
2007     7
2019     7
2011     6
2014     5
2013     5
2016     4
2023     4
2012     4
2009     4
2015     2
2022     2
2020     2
2024     1
2018     1
2010     1
2003     1
2002     1
2006     1
2017     1
2004     1
Name: count, dtype: int64

####  2) Melhorando a interação com o API
Observe dois URLs de consultas diferentes, por exemplo o URL utilizado em aula, e o URL feito no exercício anterior. Compare-os e observe as diferenças.

1. Faça uma função em Python que recebe como argumento o UF da consulta e o tipo de interessado (```'Estado'```ou ```Município```), e que devolve os dados da consulta no formato *DataFrame*.
2. Quantas solicitações para o Estado podem ser consultadas para Minas Gerais com *status* em 'Arquivado por decurso de prazo' estão registradas?
3. Qual é o município da Bahia com mais solicitações deferidas?
4. Salve um arquivo .csv com os dados de solicitações da Bahia, com interessado = 'Estado'

In [326]:
#1) Seu código aqui
def consulta(UF: str, interessado: str)-> pd.DataFrame:
    
    try:
        UF = UF.upper()
        df_tst = pd.DataFrame(requests.get('https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl').json()["items"])
        df_filtrado = df_tst[(df_tst['uf'] == UF) & (df_tst['tipo_interessado'] == interessado)]
        return df_filtrado
    except requests.exceptions.RequestException as e:
        print(f"Erro na conexão com a API: {e}")
        return pd.DataFrame()  # Retorna um DataFrame vazio em caso de erro

    except KeyError as e:
        print(f"Erro: coluna não encontrada no DataFrame - {e}")
        return pd.DataFrame()  # Retorna um DataFrame vazio em caso de erro

    except ValueError as e:
        print(f"Erro ao processar os dados: {e}")
        return pd.DataFrame()  # Retorna um DataFrame vazio em caso de erro
        
    except AttributeError as e:
        print(f"Erro de atributo: {e}")
        return pd.DataFrame()  # Retorna um DataFrame vazio em caso de erro
        
    except Exception as e:
        print(f"Ocorreu um erro inesperado: {e}")
        return pd.DataFrame()  # Retorna um DataFrame vazio em caso de erro

In [358]:
# 2) Seu código aqui
df_t = df[(df['uf'] == 'MG') & (df['status'] == 'Arquivado por decurso de prazo')]
df_t.shape[0]

65

In [374]:
# 3) Seu código aqui
df_ba = df[df['uf'] == 'BA'].reset_index()
df_ba = df_ba[df_ba['tipo_interessado'] == 'Município']
list(df_ba.interessado.value_counts().head(1).index)

['Salvador']

In [378]:
# 4) Seu código aqui
df_ba = df[df['uf'] == 'BA'].reset_index()
df_ba = df_ba[df_ba['tipo_interessado'] == 'Estado']
df_ba.to_csv('dataset-BA.csv')